## Trabajo RL Víctor Rodríguez Part
Para este trabajo intentaré entrenar a un agente que aprenda a invertir en bolsa en un '3rd party environment' de gymnassium. Específicamente gym-trading_env.

Para esto usaremos la técnica A2C. Específicamente importaré la clase A2C de la librería stable-baselines3.



In [1]:
import sys

sys.version

'3.10.13 (main, Sep 11 2023, 13:21:10) [GCC 11.2.0]'

1. Realizamos las importaciones.

In [2]:
import gymnasium as gym
import gym_trading_env

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gym_trading_env.downloader import download
from datetime import datetime, timedelta

import tensorflow as tf
import keras
from keras import layers, optimizers

from sklearn.linear_model import LinearRegression


2024-06-29 19:06:52.049876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Instanciamos el environment. Pdemos encontrar la información en: https://gym-trading-env.readthedocs.io/en/latest/#

- Descargamos y guardamos los datos del mercado en la variable df con las recomendaciones de la documentación.
- Indicamos en position las acciónes (y porcentaje de la cartera) que puede invertir en cada acción. En este caso, lo dejaremos apalancarse ya que done = True cuando la cartera vale 0. Por lo que si no se apalanca, nunca podrá acabar el episodio. Entonces pondremos el rango de acciones entre -2 (short) y 2 (long). Tampoco queremos que el actor lo apueste todo cada vez, por lo que permitiremos al bot invertir en múltiplos del 25% de la cartera.
- Iniciamos la posición en 0 (todo USD)
- El 'observation space' consiste en:
    - Observaciones estáticas (el actor no las puede cambiar): Los datos del mercado introducidos en el data frame.
    - Observaciones dinámicas (el actor las puede cambiar): Última acción del agente y la posición actual.
- Según la documentación, tenemos que añadir al data frame, columnas con el nombre feature con los inputs que queramos pasarle a la red.

In [3]:

# Download BTC/USDT historical data from Binance and stores it to directory ./data/binance-BTCUSDT-1h.pkl
download(exchange_names = ["binance"],
    symbols= ["BTC/USDT"],
    timeframe= "1h",
    dir = "data",
    since= datetime(year= 2018, month= 1, day=1),
    until = datetime(year= 2024, month= 6, day=29, hour=11),
)
# Import your fresh data


In [ ]:
df = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")

Añadimos las features recomendadas en la documentación.

In [ ]:
df["feature_close"] = df["close"].pct_change()
df["feature_open"] = df["open"]/df["close"]
df["feature_high"] = df["high"]/df["close"]
df["feature_low"] = df["low"]/df["close"]
df["feature_volume"] = df["volume"] / df["volume"].rolling(7*24).max()



Alternativamente, añadiré unas features que me parecen convenientes para representar la tendéncia de la última semana. Estas serán.

- Pendiente de la regresión lineal de la semana. Nos indica si estamos en una semana alcista o bajista.
- Máximo y mínimo de la semana. Veremos la amplitud de los precios de la semana.
- Precio de apertura de 7 días antes y precio de cierre de esta semana. Esto nos permitirá junto al resto de features aproximar la forma de la curva y ver si la tendencia es ascendente, descendente, cóncava, convexa...
- Distancia al máximo y al mínimo de la semana, para ver formas no lineales.

In [ ]:
def calcular_pendiente(datos):
    x = np.arange(len(datos)).reshape(-1, 1)
    y = datos
    modelo = LinearRegression().fit(x, y)
    pendiente = modelo.coef_[0]
    return pendiente

In [ ]:
rolling_windows = df.rolling(window=7*24)
df['feature_pendiente'] = rolling_windows['close'].apply(calcular_pendiente, raw=False)

In [ ]:
df['feature_price'] = df['close']
df['feature_initial_price'] = df['open'].shift(7*24 - 1)
df['feature_mid_price'] = df['open'].shift(7*12 - 1)
df['feature_max_week'] = rolling_windows['high'].max()
df['feature_min_week'] = rolling_windows["low"].min()
# df['feature_distance_max'] = df['date_close'] - df[df['high']== df['feature_max_week']]['date_close'].iloc[0]
# df['feature_distance_min'] = df[df["low"].rolling(7*24).max()]


# df.dropna(inplace= True)

He intentado sacar la diferencia de horas hasta la fecha pero no he podido, a cambio añado el precio a mitad semana, lo que ayuda a aproximar si es logarítmica, concava, convexa o exponencial

In [ ]:
# def find_max_index(x):
#     if len(x) > 1:
#         return x[:-1].idxmax()
#     else:
#         return x.idxmax()

# # Encontrar el índice del máximo dentro de la ventana deslizante
# rolling_windows = df.rolling(window=7*24, min_periods=1)
# max_index = rolling_windows['high'].apply(find_max_index)
# df['max_date_before'] = df.loc[max_index.values, 'date_close'].reset_index(drop=True)

In [ ]:
df

,open,high,low,close,volume,date_close,feature_close,feature_open,feature_high,feature_low,feature_volume,feature_pendiente,feature_price,feature_initial_price,feature_mid_price,feature_max_week,feature_min_week
date_open,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199,2018-01-01 01:00:00,NaN,1.013796,1.013796,0.990465,NaN,NaN,13529.01,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006,2018-01-01 02:00:00,-0.024093,1.024686,1.029753,0.996389,NaN,NaN,13203.06,NaN,NaN,NaN,NaN
2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572,2018-01-01 03:00:00,0.009628,0.990459,1.006620,0.990234,NaN,NaN,13330.18,NaN,NaN,NaN,NaN
2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030,2018-01-01 04:00:00,0.005990,0.994051,1.015007,0.991049,NaN,NaN,13410.03,NaN,NaN,NaN,NaN
2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329,2018-01-01 05:00:00,0.014242,0.987793,1.001638,0.979497,NaN,NaN,13601.01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-29 04:00:00,60762.00,60800.00,60704.46,60773.55,251.112330,2024-06-29 05:00:00,0.000190,0.999810,1.000435,0.998863,0.038236,-21.701269,60773.55,64364.12,61387.40,64546.81,58402.0
2024-06-29 05:00:00,60773.55,60788.20,60726.05,60762.02,250.059710,2024-06-29 06:00:00,-0.000190,1.000190,1.000431,0.999408,0.038076,-21.474288,60762.02,64424.21,61381.44,64546.81,58402.0
2024-06-29 06:00:00,60762.02,60857.28,60742.08,60847.75,246.690710,2024-06-29 07:00:00,0.001411,0.998591,1.000157,0.998263,0.037563,-21.218682,60847.75,64430.01,62159.09,64546.81,58402.0


In [ ]:
#Crearemos la función actualizar_grafica para que, cada nueva hora,
# la gráfica se vaya actualizando y el agente ya entrenado pueda invertir
# o desinvertir.

def actualizar_grafica(df):

    try:
        download(
            exchange_names = ["binance"],
            symbols= ["BTC/USDT"],
            timeframe= "1h",
            dir = "data",
            since= (df['date_close'].iloc[-1] - timedelta(hours=2)),
            until = (datetime.now() + timedelta(hours=1))
        )

    except:
        return df
    # new_rows = None
    new_rows = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")

    new_rows["feature_close"] = new_rows["close"].pct_change()
    new_rows["feature_open"] = new_rows["open"]/new_rows["close"]
    new_rows["feature_high"] = new_rows["high"]/new_rows["close"]
    new_rows["feature_low"] = new_rows["low"]/new_rows["close"]

    df = pd.concat([df, new_rows])
    df['feature_volume'] = df['feature_volume'].fillna(df['volume'] / df['volume'].rolling(7*24, min_periods=1).max())
    rolling_windows = df.rolling(window=7*24)
    df['feature_pendiente'] = rolling_windows['close'].apply(calcular_pendiente, raw=False)
    df['feature_price'] = df['close']
    df['feature_initial_price'] = df['open'].shift(7*24 - 1)
    df['feature_mid_price'] = df['open'].shift(7*12 - 1)
    df['feature_max_week'] = rolling_windows['high'].max()
    df['feature_min_week'] = rolling_windows["low"].min()
    df.dropna(how = 'any', inplace= True)
    return df

In [ ]:
df = actualizar_grafica(df)
df

BTC/USDT downloaded from binance and stored at data/binance-BTCUSDT-1h.pkl


,open,high,low,close,volume,date_close,feature_close,feature_open,feature_high,feature_low,feature_volume,feature_pendiente,feature_price,feature_initial_price,feature_mid_price,feature_max_week,feature_min_week
date_open,,,,,,,,,,,,,,,,,
2018-01-08 00:00:00,16218.85,16322.30,15908.06,15948.96,581.369323,2018-01-08 01:00:00,-0.012450,1.016922,1.023408,0.997436,0.346401,23.703769,15948.96,13715.65,14398.97,17176.24,12750.0
2018-01-08 01:00:00,15941.93,16001.80,15655.00,15850.07,878.804115,2018-01-08 02:00:00,-0.006200,1.005796,1.009573,0.987693,0.523624,23.521500,15850.07,13528.99,14518.95,17176.24,12750.0
2018-01-08 02:00:00,15850.08,16100.00,15830.01,16037.38,765.781916,2018-01-08 03:00:00,0.011818,0.988321,1.003905,0.987070,0.456281,23.303233,16037.38,13203.00,14501.05,17176.24,12750.0
2018-01-08 03:00:00,16030.01,16031.99,15514.99,15670.37,948.188780,2018-01-08 04:00:00,-0.022885,1.022950,1.023077,0.990084,0.564966,23.027420,15670.37,13330.26,14367.99,17176.24,12750.0
2018-01-08 04:00:00,15632.48,15840.00,15510.00,15641.01,638.906874,2018-01-08 05:00:00,-0.001874,0.999455,1.012722,0.991624,0.380684,22.756556,15641.01,13434.98,14469.97,17176.24,12750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-29 13:00:00,61045.43,61182.70,61010.00,61102.98,517.966610,2024-06-29 14:00:00,0.000943,0.999058,1.001305,0.998478,0.078869,-18.741506,61102.98,64287.96,62028.01,64521.00,58402.0
2024-06-29 14:00:00,61102.98,61160.00,60949.67,61018.01,542.430960,2024-06-29 15:00:00,-0.001391,1.001393,1.002327,0.998880,0.082594,-18.400452,61018.01,64307.09,61936.07,64521.00,58402.0
2024-06-29 15:00:00,61018.01,61106.00,60956.79,61106.00,364.187030,2024-06-29 16:00:00,0.001442,0.998560,1.000000,0.997558,0.055454,-18.029443,61106.00,64321.29,61712.96,64521.00,58402.0


In [ ]:
# df['feature_mid_price'] = df['open'].shift(7*12 - 1)
# df.to_csv('data/data_frame.csv')

In [ ]:
#Definimos el espacio de acciones y creamos el entorno
positions = list(np.arange(-2, 2.01, 0.25))
positions[5] = 0 #La función crea un número muy cercano a 0, pero no es 0
env = gym.make("TradingEnv",
               df=df, positions = positions,
               initial_position= 0,
               borrow_interest_rate= 0.0003/100)

In [ ]:
s0, info= env.reset()
s0, info

(array([-1.2450132e-02,  1.0169221e+00,  1.0234084e+00,  9.9743557e-01,
         3.4640121e-01,  2.3703770e+01,  1.5948960e+04,  1.3715650e+04,
                   nan,  1.7176240e+04,  1.2750000e+04,  0.0000000e+00,
         0.0000000e+00], dtype=float32),
 {'idx': 0,
  'step': 0,
  'date': numpy.datetime64('2018-01-08T00:00:00.000000000'),
  'position_index': 5,
  'position': 0,
  'real_position': 0,
  'data_volume': 581.369323,
  'data_date_close': Timestamp('2018-01-08 01:00:00'),
  'data_low': 15908.06,
  'data_high': 16322.3,
  'data_open': 16218.85,
  'data_close': 15948.96,
  'portfolio_valuation': 1000.0,
  'portfolio_distribution_asset': 0,
  'portfolio_distribution_fiat': 1000.0,
  'portfolio_distribution_borrowed_asset': 0,
  'portfolio_distribution_borrowed_fiat': 0,
  'portfolio_distribution_interest_asset': 0,
  'portfolio_distribution_interest_fiat': 0,
  'reward': 0})

### 3. Creamos un ejercicio aleatorio según las recomendaciones de la documentación.

In [ ]:
done, truncated = False, False
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)

    #Esto es para renderizar los trades, pero es muy lento, por lo que de momento lo comento.
    # env.unwrapped.save_for_render(dir = "render_logs")

Market Return : 282.43%   |   Portfolio Return : -93.40%   |   


In [ ]:
info

{'idx': 56632,
 'step': 56632,
 'date': numpy.datetime64('2024-06-29T17:00:00.000000000'),
 'position_index': 16,
 'position': 2.0,
 'real_position': 2.000537292223384,
 'data_volume': 41.15702,
 'data_date_close': Timestamp('2024-06-29 18:00:00'),
 'data_low': 60964.85,
 'data_high': 61009.91,
 'data_open': 61009.91,
 'data_close': 60993.62,
 'portfolio_valuation': 66.0051403648398,
 'portfolio_distribution_asset': 0.0021649107690001183,
 'portfolio_distribution_fiat': 0,
 'portfolio_distribution_borrowed_asset': 0,
 'portfolio_distribution_borrowed_fiat': 66.04040629224234,
 'portfolio_distribution_interest_asset': 0.0,
 'portfolio_distribution_interest_fiat': 0.000198121218876727,
 'reward': -0.0005356514880103416}

In [ ]:
#Lo mismo

# from gym_trading_env.renderer import Renderer
# renderer = Renderer(render_logs_dir="render_logs")
# renderer.run()

### 4. Definimos el modelo y los hiperparámetros.

In [ ]:
def create_q_model():
    inputs = layers.Input(shape=(*s0.shape, 1))

    layer1 = layers.Dense(32, activation='relu')(inputs)
    layer2 = layers.Dense(64, activation='relu')(layer1)
    layer3 = layers.Dense(128, activation='relu')(layer2)
    layer4 = layers.Dense(32, activation='relu')(layer3)


    action = layers.Dense(env.action_space.n, activation='linear')(layer4)

    return keras.Model(inputs=inputs, outputs=action)

In [ ]:
decay = 0.9999
epsilon_ini = 1.0
epsilon_end = 0.05
learning_rate = 0.001
gamma = 0.999
max_memory_size = int(10e5)
batch_size = 16

In [ ]:
q_model = create_q_model()

optimizer = optimizers.Adam(learning_rate=learning_rate)
q_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13, 1)]           0         
                                                                 
 dense_10 (Dense)            (None, 13, 32)            64        
                                                                 
 dense_11 (Dense)            (None, 13, 64)            2112      
                                                                 
 dense_12 (Dense)            (None, 13, 128)           8320      
                                                                 
 dense_13 (Dense)            (None, 13, 32)            4128      
                                                                 
 dense_14 (Dense)            (None, 13, 17)            561       
                                                                 
Total params: 15,185
Trainable params: 15,185
Non-trainable

In [ ]:
def get_best_action(state):
    q_state = q_model(state)
    return np.argmax(q_state)

In [ ]:
s0 = np.expand_dims(s0, axis=0)
q0 = q_model(s0)
q0

<tf.Tensor: shape=(1, 13, 17), dtype=float32, numpy=
array([[[-2.38993031e-04,  1.12616341e-03, -1.67459046e-04,
          5.59679058e-04,  1.03628228e-03, -1.05377717e-03,
         -1.29365374e-03, -1.07793210e-04, -5.80006570e-04,
         -1.93187981e-04, -8.63178459e-04, -8.90745723e-05,
          1.53131434e-03,  1.07775594e-03,  6.54518022e-04,
         -2.31115753e-03,  1.85020588e-04],
        [-3.82853933e-02,  1.02421768e-01, -3.41919437e-02,
          1.89169329e-02, -1.49816712e-02,  5.87383434e-02,
         -4.77586612e-02, -2.79653408e-02,  4.96413596e-02,
          3.42851058e-02,  1.45773515e-02, -5.04627749e-02,
          8.90824497e-02,  1.14603855e-01,  2.70627383e-02,
         -8.35711509e-02, -7.23320320e-02],
        [-3.85296196e-02,  1.03075080e-01, -3.44100595e-02,
          1.90376136e-02, -1.50772221e-02,  5.91130331e-02,
         -4.80632856e-02, -2.81437077e-02,  4.99580093e-02,
          3.45038176e-02,  1.46703301e-02, -5.07846437e-02,
          8.9650698